# 电磁式阻尼器振动控制训练
本notebook用于训练基于GRUA网络架构的TD3算法，用于电磁式阻尼器的振动控制，

### 主要特点
- 使用TD3（Twin Delayed Deep Deterministic Policy Gradient）作为基础算法
- 输入时间序列，支持延时
- 针对电磁式阻尼器系统的振动控制优化

## 训练环境与项目日志初始化

In [1]:
import numpy as np
import torch
import json  # 导入json模块
import logging

from data import plot_data, format_dict, make_dirs

train_model = True  # 是否训练模型
controller_type = "TD3"  # 控制器类型：TD3 或 PPO
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from datetime import datetime
import os
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
project_name = input("请输入加载/创建项目的名称 (父目录: .\\savedata) ").strip()# 创建保存模型的基础目录
project_path, ckpt_dir, plt_dir = make_dirs(project_name)
file_path = os.path.join(project_path, f'training_{current_time}.log')
logging.basicConfig(filename=file_path, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s') # 设置日志格式
logging.info("## 当前时间: %s", datetime.now())
logging.info("项目保存目录: %s", project_path)
logging.info("日志文件: %s", file_path)
logging.info("训练模式: %s", train_model)
logging.info("使用设备: %s", device)
print(f"日志文件: {file_path}")

日志文件: savedata\20260108下午mlp无延迟0-初速度条件，观测维度少，不包括delay\training_20260109_160511.log


## 训练参数

In [3]:
from fx import tolerance_reward, zero

ENV_PARAMS = {
    'Ts': 0.001,  # 环境时间步长
    'T': 1.0,     # 每回合总时间
    'state0': np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0]),  # 初始状态
    'obs_indices': [0, 2, 3, 5],  # 观测状态索引
    'x1_limit': 0.03,  # 状态 x1 的限制
    'use_dt_noise': False,  # 是否使用时间步长噪声
    'dt_noise_std': 0.1,  # 时间步长噪声标准差
    'delay_enabled': False,  # 是否启用动作延迟
    'delay_mean_steps': 0,  # 延迟均值步数
    'delay_std_steps': 0,   # 延迟标准差步数
    'include_dt_in_obs': False,  # 是否在观测中包含时间步长
    'include_delay_in_obs': False,  # 是否在观测中包含延迟
    'z_func': zero,  # 状态惩罚函数
    'r_func': tolerance_reward(1e-3),  # 奖励函数
    'f_func': zero,  # 外部激励函数
}

TD3_PARAMS = {
    # 创建神经网络的参数
    'arch': 'mlp',  # 网络架构类型
    'norm': False,      # 是否使用归一化
    'simple_nn': False,  # 是否使用简单神经网络
    'state_dim': len(ENV_PARAMS['obs_indices']) + int(ENV_PARAMS['include_dt_in_obs']) + int(ENV_PARAMS['include_delay_in_obs']),  # 状态维度
    'action_dim': 1,  # 动作维度
    'action_bound': 5,  # 动作边界
    'gru_hidden': 64,  # GRU 隐藏层大小
    'gru_layers': 1,   # GRU 层数
    'hidden_dim': 128,  # 全连接层隐藏维度
    'seq_len': 36,     # 预测前序列长度
    'fc_seq_len': 4,   # 预测器预测序列长度
    # 优化器参数
    'actor_lr': 2e-06,  # Actor 学习率
    'critic_lr': 1e-05,  # Critic 学习率
    'clip_grad': 1.0,  # 梯度裁剪值
    'tau': 0.002,      # 软更新参数
    # TD3 算法参数
    'gamma': 0.99,     # 折扣因子
    'policy_noise': 0.2,  # 策略噪声标准差
    'noise_clip': 0.5,   # 噪声裁剪值
    'policy_freq': 3,   # 策略延迟频率 
}

TRAIN_PARAMS = {
    'explore_noise_trend': 'exp',  # 探索噪声初始标准差
    'n_episodes': 2000,    # 训练回合数
    'batch_size': 128,    # 训练批次大小
    'min_buffer_size': 5_000,  # 最小缓冲区大小
    'save_interval': 5,  # 模型保存间隔回合数
    'resume': True,      # 是否从检查点恢复训练
}

logging.info("环境参数:\n%s", json.dumps(format_dict(ENV_PARAMS), indent=4, ensure_ascii=False))
logging.info("TD3 参数:\n%s", json.dumps(TD3_PARAMS, indent=4, ensure_ascii=False))
logging.info("训练参数:\n%s", json.dumps(TRAIN_PARAMS, indent=4, ensure_ascii=False))

## 初始化环境、控制器、经验池

In [4]:
from env import build_env
test_env = build_env(ENV_PARAMS)
test_recorder = test_env.run_episode(record=True)
x_values=test_recorder.as_numpy(keys='time_history').reshape(-1, 1)
y_values=test_recorder.as_numpy(keys='state_history')[:,[0,1,2,3,4,5]]
plot_data(x_values=x_values, y_values=y_values, legends=[('吸振器位移',),('主结构位移',),('吸振器速度',),('主结构速度',),('吸振器加速度',),('主结构加速度',)], sub_shape=(3,2),
          sub_group=[(0,),(3,),(1,),(4,),(2,),(5,)],plot_title='测试环境-无控制响应',legend_loc='upper right')

In [5]:
from controller import build_controller

controller = build_controller(TD3_PARAMS, type=controller_type)

logging.info("Actor 网络结构:\n%s", controller.actor)
logging.info("Critic 网络结构:\n%s", controller.critic1)

In [6]:
from buffer import ReplayBuffer
replay_buffer = ReplayBuffer(state_dim=TD3_PARAMS['state_dim'],
                             capacity=1000_000,
                             seq_len=TD3_PARAMS['seq_len'])

## 训练

In [9]:
if train_model:
    from train import train
    history = train(project_name=project_name, 
                    env=build_env(ENV_PARAMS), controller=controller, buffer=replay_buffer, 
                    explore_noise_trend=TRAIN_PARAMS['explore_noise_trend'], n_episodes=TRAIN_PARAMS['n_episodes'],
                    batch_size=TRAIN_PARAMS['batch_size'], min_buffer_size=TRAIN_PARAMS['min_buffer_size'], 
                    save_interval=TRAIN_PARAMS['save_interval'], resume=TRAIN_PARAMS.get('resume', False))


找到以下检查点文件:
1. 0108_135253_ep5ckpt.pth
2. 0108_140300_ep5ckpt.pth
3. 0108_140331_ep10ckpt.pth
4. 0108_140402_ep15ckpt.pth
5. 0108_140434_ep20ckpt.pth
6. 0108_140507_ep25ckpt.pth
7. 0108_140541_ep30ckpt.pth
8. 0108_140615_ep35ckpt.pth
9. 0108_140650_ep40ckpt.pth
10. 0108_140728_ep45ckpt.pth
11. 0108_140810_ep50ckpt.pth
12. 0108_140852_ep55ckpt.pth
13. 0108_140931_ep60ckpt.pth
14. 0108_141013_ep65ckpt.pth
15. 0108_141052_ep70ckpt.pth
16. 0108_141132_ep75ckpt.pth
17. 0108_141212_ep80ckpt.pth
18. 0108_141257_ep85ckpt.pth
19. 0108_141342_ep90ckpt.pth
20. 0108_141428_ep95ckpt.pth
21. 0108_141514_ep100ckpt.pth
22. 0108_141601_ep105ckpt.pth
23. 0108_141648_ep110ckpt.pth
24. 0108_141737_ep115ckpt.pth
25. 0108_141826_ep120ckpt.pth
26. 0108_141918_ep125ckpt.pth
27. 0108_142011_ep130ckpt.pth
28. 0108_142107_ep135ckpt.pth
29. 0108_142203_ep140ckpt.pth
30. 0108_142259_ep145ckpt.pth
31. 0108_142356_ep150ckpt.pth
32. 0108_142456_ep155ckpt.pth
33. 0108_142556_ep160ckpt.pth
34. 0108_142658_ep165ckpt.pth

 38%|███▊      | 456/1200 [4:50:41<7:54:17, 38.25s/it] 


KeyboardInterrupt: 